In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [10]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [11]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [12]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [13]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [14]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [15]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

#### condense bottom 8 classes into single class

In [ ]:
# df_train['genre_id'] = df_train['genre_id'].apply(lambda x: 5 if x>=5 else x).values

In [ ]:
# df_train['genre_id'].value_counts()

### Create Folds

In [ ]:
df_train['fold'] = 0

In [ ]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [ ]:
df_train.head()

In [ ]:
df_test.head()

#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
#     [torchvision.transforms.Resize((128,128))])
    [torchvision.transforms.CenterCrop((64,960))],)
#    torchaudio.transforms.TimeMasking(48)])


In [ ]:
import torchaudio

In [ ]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
#                      tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds',mode="per_example",p_mode="per_example",sample_rate=16000),
#                     tA.BandPassFilter(),
                    tA.TimeInversion(p=0.5,p_mode="per_example",sample_rate=16000),
#                     tA.PitchShift()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.5,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [ ]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 64
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/train/{filename}')
        else:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/test/{filename}')
            
        return spec


    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [ ]:
# a,b = next(iter(POGFiles(df_train)))
# a.shape

In [ ]:
a,b = next(iter(POGFiles(df_train,transform=None)))
#` a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=576,N_FFT=512)))
a.shape

#### FastAI model

In [ ]:
df_train['genre_id'].nunique()

In [ ]:
import timm
timm.list_models()

In [ ]:
timm.create_model('convnext_base',in_chans=1)

In [ ]:
df_train['genre_id'].value_counts()

In [ ]:
df_train.head()

In [ ]:
NCLASS = df_train['genre_id'].nunique()

In [ ]:
NCLASS

#### Mixup

In [ ]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE_MIXUP'
        kernel_type_prev = 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE'        
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit =  CrossEntropyLossFlat() #FocalLoss() #LabelSmoothingCrossEntropy(0.01)
        batch_size = 24
        n_epochs = 15

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('densenet121d', pretrained=False, in_chans=1)
        model.features.conv0.stride = (1,1)
        model.classifier = nn.Linear(1024,NCLASS)
    
        f1_score = F1Score(average="micro")
        
        
        
        learn = Learner(dls, model,loss_func=crit,metrics=[f1_score],cbs=[MixUp(0.25),SaveModelCallback('f1_score', every_epoch=True),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()

        weights = torch.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_prev}/fold_{fold_num}.pth')['model']
        learn.model.load_state_dict(weights,strict=False)

        learn.fit_one_cycle(n_epochs, 5e-4, wd=1e-03)
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [ ]:
if __name__=='__main__':
    run()

### OOF

In [ ]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE_MIXUP'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

In [ ]:
OOFOut['ID']

In [ ]:
OOFOut.tail()

In [ ]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [ ]:
OOFOut.head()

In [ ]:
OOFOut['genre_id'].max()

In [ ]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [ ]:
OOF_preds

In [ ]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

In [ ]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

### Mode based sub

In [ ]:
kernel_type = 'ecaresnext50t_32x4d_CQT_512_64_MIXUP'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub: check OOF Score

In [16]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160',
                'resnext50d_32x4d_MonoSpec_Hop448_Mels160',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE_MIXUP',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160_NoisyTrain',
#                 'resnest101e_MonoSpec_Hop512_Mels128'
#                 'nf_resnet50_MonoSpec_Hop448_Mels160',
#                     'seresnet50t_MonoSpec_Hop448_Mels160',
#                     'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
#                     'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                     'tf_efficientnet_b1_ns_MonoSpec_Hop576_Mels128',

#                     'densenet121_CQT',
#                     'ecaresnext50t_32x4d_CQT_512_64',
                    'ecaresnext50t_32x4d_CQT_512_64',
                    'ecaresnext50t_32x4d_CQT_512_64_MIXUP',
#                     'resnest50d_CQT_512_64',
#                     'ecaresnext50t_32x4d_CQT_448_84',
                    'ecaresnext50t_32x4d_CQT_448_84_TUNE',
#                     'regnetz_d32_MonoSpec_Hop448_Mels160',
                    'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels64'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

In [92]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [

                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160',

                'resnext50d_32x4d_MonoSpec_Hop448_Mels160',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels64',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels64_TUNE',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels64_Noisy',
                'resnext50d_32x4d_MonoSpec_Hop448_Mels64_Noisy',

    
                'ecaresnext50t_32x4d_CQT_512_64',
                'ecaresnext50t_32x4d_CQT_512_64_MIXUP',
                'ecaresnext50t_32x4d_CQT_448_84_TUNE',
                'ecaresnext50t_32x4d_CQT_512_64_Noisy',

               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

0.6257283504118947

In [88]:
from sklearn.metrics import f1_score

out1 = pd.DataFrame()
for kernels in ['eb1_MonoSpec_Hop448_Mels160',
                'densenet121d_MonoSpec_Hop448_Mels64_Noisy',
                'densenet121d_MonoSpec_Hop448_Mels64_Noisy',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'densenet121d_MonoSpec_Hop448_Mels64_Noisy',
                'regnetz_d32_MonoSpec_Hop448_Mels160',
                'ecaresnet50d_CQT_512_64',
                'resnest50d_CQT_512_64',
                'ecaresnext26t_32x4d_CQT_512_64_Noisy',
                'efficientnet_b1_CQT_512_64',
                'efficientnet_b2_CQT_512_64_Noisy',
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out1 = pd.concat([out1,OOF_preds],1)
    out1['genre_id'] = OOFOut['genre_id']
    
out1['preds'] = out1.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out1['genre_id'],out1['preds'],average="micro")

0.5975487241310026

In [77]:
pd.crosstab(out1['preds'],out['preds'])

preds,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
preds,,,,,,,,,,,,,,,,,,
0,3893,58,13,67,93,3,125,31,44,41,90,0,0,4,0,6,0,0
1,62,2877,337,81,8,52,4,47,105,20,5,3,0,9,20,3,0,5
2,8,387,1988,4,4,0,2,1,10,4,0,0,0,0,3,0,0,1
3,81,73,10,1765,7,21,11,51,14,12,33,17,5,12,0,1,7,0
4,91,9,2,8,1537,0,2,2,8,8,1,0,0,0,0,1,5,0
5,4,55,1,25,1,1139,0,16,28,15,0,7,0,8,6,0,2,0
6,80,4,1,16,2,0,727,1,2,1,1,0,0,1,0,0,0,0
7,41,38,0,56,1,17,2,712,5,1,43,20,0,8,0,0,0,0
8,27,30,4,6,5,22,0,1,216,4,1,0,0,1,1,4,0,0


In [78]:
pd.crosstab(out1['preds'],out1['genre_id'])

genre_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
preds,,,,,,,,,,,,,,,,,,,
0,2294,175,55,191,308,19,442,138,173,121,469,2,0,29,0,43,0,3,6
1,134,1779,793,166,17,154,8,128,280,37,19,8,0,35,47,14,0,19,0
2,15,668,1593,19,9,9,3,16,51,12,0,0,0,2,9,2,2,2,0
3,165,160,62,1101,20,61,45,167,94,31,96,22,5,43,2,1,36,6,3
4,152,16,15,34,1360,8,11,3,22,26,12,0,0,0,0,5,3,6,1
5,15,109,20,60,4,859,1,31,99,39,3,8,1,12,29,3,7,7,0
6,106,6,3,30,8,1,652,2,11,0,16,0,0,1,0,0,0,0,0
7,77,62,2,78,4,34,7,489,24,2,85,39,0,35,1,2,0,2,1
8,35,52,12,18,10,23,2,4,146,9,2,0,0,0,2,5,0,1,1


In [79]:
from sklearn.metrics import classification_report

print(classification_report(out1['genre_id'],out1['preds']))

              precision    recall  f1-score   support

           0       0.51      0.75      0.61      3071
           1       0.49      0.57      0.53      3095
           2       0.66      0.62      0.64      2582
           3       0.52      0.61      0.56      1800
           4       0.81      0.77      0.79      1756
           5       0.66      0.71      0.68      1214
           6       0.78      0.55      0.65      1181
           7       0.52      0.47      0.49      1044
           8       0.45      0.15      0.23       945
           9       0.66      0.63      0.65       814
          10       0.45      0.10      0.17       796
          11       0.78      0.82      0.80       495
          12       0.97      0.98      0.98       408
          13       0.69      0.34      0.46       306
          14       0.81      0.27      0.40       142
          15       0.67      0.06      0.12        94
          16       0.46      0.43      0.44        94
          17       1.00    

In [80]:
print(classification_report(out['genre_id'],out['preds']))

              precision    recall  f1-score   support

           0       0.53      0.75      0.62      3071
           1       0.55      0.63      0.58      3095
           2       0.71      0.65      0.68      2582
           3       0.54      0.63      0.58      1800
           4       0.83      0.78      0.81      1756
           5       0.68      0.71      0.69      1214
           6       0.78      0.58      0.67      1181
           7       0.53      0.46      0.50      1044
           8       0.45      0.21      0.29       945
           9       0.75      0.69      0.72       814
          10       0.46      0.17      0.25       796
          11       0.80      0.83      0.82       495
          12       0.97      0.99      0.98       408
          13       0.73      0.45      0.56       306
          14       0.85      0.46      0.60       142
          15       0.46      0.13      0.20        94
          16       0.54      0.50      0.52        94
          17       1.00    

In [81]:
outDF = out.copy()

f1_score(outDF['genre_id'],outDF['preds'],average="micro")

0.6257283504118947

In [82]:
outDF = out.copy()
outDF.loc[out1['preds']==1,'preds'] = 1

f1_score(outDF['genre_id'],outDF['preds'],average="micro")

0.625075346594334

In [53]:
outDF[out1['preds']==1]['preds']

0        1
5        1
14       1
15       1
36       0
        ..
19885    1
19887    3
19890    1
19894    1
19905    1
Name: preds, Length: 3622, dtype: int32

### Mode based sub

In [ ]:
kernel_type

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'densenet121d_MonoSpec_Hop448_Mels160_FTUNE'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_4bestmodels.csv',index=False)

In [ ]:
kernel_type

### PL

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'densenet121d_MonoSpec_Hop448_Mels160_FTUNE'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
mode = out.drop('song_id',1).mode(1)[0].values

In [ ]:
(out.iloc[0,1:]==mode[0]).sum()

In [ ]:
PL_song = pd.DataFrame()
ct = 0
for j in range(out.shape[0]):
    m = mode[j]
    if (out.iloc[j,1:]==m).sum()==20:
        PL_song.loc[ct,'song_id'] = out.loc[j,'song_id'].astype(int)
        PL_song.loc[ct,'genre_id'] = m.astype(int)
    ct += 1

In [ ]:
PL_song

In [ ]:
out[out['song_id']==10207.0]

In [ ]:
PL_song['song_id'] = PL_song['song_id'].astype(int)
PL_song['genre_id'] = PL_song['genre_id'].astype(int)

In [ ]:
PL_song.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/4_Ense_PL.csv',index=False)

### End ###